# Coupon-Bond Guaranteed on Mainnet with Reference Scripts

**Executive Summary**

This example demonstrates the use of `marlowe-cli` with Plutus V2 reference scripts for the two Marlowe validators on `mainnet`. It is a three-party contract for a guaranteed loan, where a *guarantor* deposits principal plus interest before a *lender* makes the loan to the *borrower*. As the borrower pays principal and/or interest, the guarantor receives the relevant part of their guarantee back while the lender receives the repayment. The total fee (Plutus and non-Plutus) paid in this example is approximately 10 Ada.

**Highlights**

- Running Marlowe on `mainnet`.
- Automatically computing validity intervals for transactions.
- Auto-balancing of transactions.
- Use of reference scripts for the Marlowe validators.
- Compute total fees paid over the course of the transaction.

**The Contract**

We use the example "Coupon Bond Guaranteed" from [Marlowe Playground](https://marlowe-playground-staging.plutus.aws.iohkdev.io/).

![Coupon Bond Guaranteed](coupon-bond-guaranteed.png)

## Preliminaries

Record version numbers

In [1]:
marlowe-cli --version

marlowe-cli 0.0.8.6 «mainnet»


In [2]:
cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [3]:
git rev-parse HEAD

6ac3aced0258ac23aec5e0b19f108df102874f84


Set the location of keys.

In [4]:
TREASURY=/extra/iohk/networks/treasury

### Select Network

Use `mainnet` at your own risk, because Marlowe has not yet completed its audit!

In [5]:
MAGIC=(--mainnet)

In [6]:
export CARDANO_NODE_SOCKET_PATH=/extra/iohk/networks/mainnet/node.socket

### Constants

In [7]:
ADA=1000000

In [8]:
SECOND=1000
MINUTE=$((60 * SECOND))
HOUR=$((60 * MINUTE))
DAY=$((24 * HOUR))

### Utility Functions

Fund an address from the faucet.

In [9]:
function faucet {
  marlowe-cli util faucet "${MAGIC[@]}" \
                          --required-signer "$FAUCET_SKEY" \
                          --faucet-address "$FAUCET_ADDR" \
                          --lovelace "$2" \
                          --out-file /dev/null \
                          --submit 600 \
                          "$1"
}

Find a UTxO with the specified lovelace.

In [10]:
function lovelace-utxo {
  marlowe-cli util select "${MAGIC[@]}" --lovelace-only "$2" "$1"
}

Find the UTxO for a role token at an address.

In [11]:
function role-token-utxo {
  marlowe-cli util select "${MAGIC[@]}" --asset-only "$2" "$1"
}

List the UTxOs at the guarantor's address.

In [12]:
function query-guarantor {
  echo
  echo "UTxOs at the guarantor's address:"
  cardano-cli query utxo "${MAGIC[@]}" --address "$GUARANTOR_ADDR"
}

List the UTxOs at the lender's address.

In [13]:
function query-lender {
  echo
  echo "UTxOs at the lender's address:"
  cardano-cli query utxo "${MAGIC[@]}" --address "$LENDER_ADDR"
}

List the UTxOs at the borrower's address.

In [14]:
function query-borrower {
  echo
  echo "UTxOs at the borrower's address:"
  cardano-cli query utxo "${MAGIC[@]}" --address "$BORROWER_ADDR"
}

List the UTxOs at the contract address.

In [15]:
function query-marlowe {
  echo
  echo "UTxOs at the contracts's address:"
  cardano-cli query utxo "${MAGIC[@]}" --address "$(jq -r '.tx.marloweValidator.address' coupon-bond-guaranteed-1.marlowe)" | sed -n -e "1,2p;/^$1/p"
}

List the UTxOs at the payout address.

In [16]:
function query-payout {
  echo
  echo "UTxOs at the payout address:"
  cardano-cli query utxo "${MAGIC[@]}" --address "$(jq -r '.tx.rolesValidator.address' coupon-bond-guaranteed-1.marlowe)" | sed -n -e "1,2p;/^$1/p"
}

List the UTxOs at the faucet address.

In [17]:
function query-faucet {
  cardano-cli query utxo "${MAGIC[@]}" --address "$FAUCET_ADDR" --out-file /dev/stdout \
  | jq '. | to_entries | [ .[] | .value.value.lovelace ] | add'
}

Compute the start of the validity interval.

In [18]:
function invalid-before {
  jq -r .tx.state.minTime "coupon-bond-guaranteed-$1.marlowe"
}

Compute an end to the validity interval.

In [19]:
function invalid-hereafter {
  echo "$(($(date -u +%s) * SECOND + 20 * MINUTE))"
}

Show the accounts internal to the contract.

In [20]:
function show-accounts {
  jq '.tx.state.accounts' "coupon-bond-guaranteed-$1.marlowe" | json2yaml
}

Show the contract.

In [21]:
function show-contract {
  jq '.tx.contract' "coupon-bond-guaranteed-$1.marlowe" | json2yaml
}

### Setup the faucet.

In [22]:
FAUCET_SKEY=$TREASURY/payment.skey
FAUCET_ADDR=$(cat $TREASURY/payment.mainnet.address)
echo "$FAUCET_ADDR"

addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h


View the UTxOs at the faucet.

In [23]:
FAUCET_INITIAL_FUNDS=$(query-faucet)
echo "The faucet starts with $(echo scale=6\;$FAUCET_INITIAL_FUNDS/1000000 | bc) Ada."

The faucet starts with 240.018586 Ada.


### Participants

#### The Guarantor

In [24]:
GUARANTOR_ROLE=Guarantor

GUARANTOR_SKEY="$TREASURY/john-fletcher.skey"
GUARANTOR_VKEY="$TREASURY/john-fletcher.vkey"

GUARANTOR_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$GUARANTOR_VKEY" )
echo "$GUARANTOR_ADDR"

addr1vywt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0czta9gx


Fund the guarantor's address.

In [25]:
faucet "$GUARANTOR_ADDR" "$((50 * ADA))"

92c47b305bbfc412fbb3eeefd23b40f959bc6afbcde08ffd43457cc5c7de6f5a


#### The Lender

In [26]:
LENDER_ROLE=Lender

LENDER_SKEY="$TREASURY/thomas-kyd.skey"
LENDER_VKEY="$TREASURY/thomas-kyd.vkey"

LENDER_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$LENDER_VKEY" )
echo "$LENDER_ADDR"

addr1v87n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlchhlqt3


Fund the lender's address.

In [27]:
faucet "$LENDER_ADDR" "$((40 * ADA))"

818aa22fdc619618ad3957f1a9a89ec2ca9e3e3da76ec4c01061755d271ae4ca


#### The Borrower

In [28]:
BORROWER_ROLE=Borrower

BORROWER_SKEY="$TREASURY/francis-beaumont.skey"
BORROWER_VKEY="$TREASURY/francis-beaumont.vkey"

BORROWER_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$BORROWER_VKEY" )
echo "$BORROWER_ADDR"

addr1vxzpzll6gsl9npf8wfhk2zg8sy2we50jcqc7w8w46gua2pqmkvj99


Fund the borrower's address.

In [29]:
faucet "$BORROWER_ADDR" "$((30 * ADA))"

476d403adc1bcd371712ada727ce2ffc93e37c7c438b650fc9c590c3647f648d


### Time

In [30]:
NOW=$(($(date -u +%s) * SECOND))
echo "$NOW"

1664710947000


In [31]:
TIP=$(cardano-cli query tip "${MAGIC[@]}" | jq -r '.slot')
echo "$TIP"

73144652


### Role Tokens

The guarantor mints the role tokens.

In [32]:
MINT_EXPIRES=$((TIP + 1000000))
echo "$MINT_EXPIRES"

74144652


In [33]:
ROLES_CURRENCY=$(
marlowe-cli util mint "${MAGIC[@]}" \
                      --required-signer "$GUARANTOR_SKEY" \
                      --change-address "$GUARANTOR_ADDR" \
                      --expires "$MINT_EXPIRES" \
                      --out-file /dev/null \
                      --submit 600 \
                      "$GUARANTOR_ROLE" "$LENDER_ROLE" "$BORROWER_ROLE"\
)
GUARANTOR_TOKEN="$ROLES_CURRENCY.$GUARANTOR_ROLE"
LENDER_TOKEN="$ROLES_CURRENCY.$LENDER_ROLE"
BORROWER_TOKEN="$ROLES_CURRENCY.$BORROWER_ROLE"


Fee: Lovelace 181517
Size: 452 / 16384 = 2%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%


Here are the role tokens:

In [34]:
echo "Guarantor token: $GUARANTOR_TOKEN"
echo "Lender token: $LENDER_TOKEN"
echo "Borrower token: $BORROWER_TOKEN"

Guarantor token: 346ec66c37f28ef21ee262eaf41dfe3605aa3344dd236a6003cb606e.Guarantor
Lender token: 346ec66c37f28ef21ee262eaf41dfe3605aa3344dd236a6003cb606e.Lender
Borrower token: 346ec66c37f28ef21ee262eaf41dfe3605aa3344dd236a6003cb606e.Borrower


Distribute the role tokens to the lender and borrower.

In [35]:
marlowe-cli transaction simple "${MAGIC[@]}" \
                               --tx-in "$(lovelace-utxo $GUARANTOR_ADDR $((15*ADA)))" \
                               --tx-in "$(role-token-utxo $GUARANTOR_ADDR $BORROWER_TOKEN)" \
                               --tx-in "$(role-token-utxo $GUARANTOR_ADDR $LENDER_TOKEN)" \
                               --tx-out "$BORROWER_ADDR+$((2*ADA))+1 $BORROWER_TOKEN" \
                               --tx-out "$LENDER_ADDR+$((2*ADA))+1 $LENDER_TOKEN" \
                               --required-signer "$GUARANTOR_SKEY" \
                               --change-address "$GUARANTOR_ADDR" \
                               --out-file /dev/null \
                               --submit 600

1eea316fb8f9c11b3640fde49765225408153f26cfe861218a556322457ea403


### UTxOs

Check the UTxOs at the parties' addresses.

In [36]:
query-guarantor
query-lender
query-borrower


UTxOs at the guarantor's address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
1eea316fb8f9c11b3640fde49765225408153f26cfe861218a556322457ea403     0        44586248 lovelace + TxOutDatumNone
b8d8a9827ba3309e7d7f5dfe8e936680d86cb22b5d2e6a60c89a988402bfe39c     1        1047330 lovelace + 1 346ec66c37f28ef21ee262eaf41dfe3605aa3344dd236a6003cb606e.47756172616e746f72 + TxOutDatumNone

UTxOs at the lender's address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
1eea316fb8f9c11b3640fde49765225408153f26cfe861218a556322457ea403     2        2000000 lovelace + 1 346ec66c37f28ef21ee262eaf41dfe3605aa3344dd236a6003cb606e.4c656e646572 + TxOutDatumNone
818aa22fdc619618ad3957f1a9a89ec2ca9e3e3da76ec4c01061755d271ae4ca     1        40000000 lovelace + 

Here is the decoding of the role names:

In [37]:
for x in 47756172616e746f72 4c656e646572 426f72726f776572
do
  echo -n $x | tr '[:lower:]' '[:upper:]' | basenc --decode --base16
  echo " = $x"
done

Guarantor = 47756172616e746f72
Lender = 4c656e646572
Borrower = 426f72726f776572


### Publish the Reference Scripts for the Marlowe Validators

We publish the the faucet address, so that the min-Ada for the reference scripts can be reclaimed later.

In [38]:
marlowe-cli transaction publish "${MAGIC[@]}" \
                                --change-address "$FAUCET_ADDR" \
                                --required-signer "$FAUCET_SKEY" \
                                --at-address "$FAUCET_ADDR" \
                                --out-file /dev/null \
                                --submit 600


Fee: Lovelace 843893
Size: 15405 / 16384 = 94%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%

Marlowe script published at address: addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h

Marlowe script hash: "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0"

Marlowe ref script UTxO min ADA: Lovelace 12434350

Payout script published at address: addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h

Payout script hash: "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3"

Payout ref script UTxO min ADA: Lovelace 12434350


Check that the publication occurred.

In [39]:
marlowe-cli transaction find-published "${MAGIC[@]}" \
                                       --at-address "$FAUCET_ADDR"


Searching for reference script at address: addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h

Expected reference script hash: "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0"

Searching for reference script at address: addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h

Expected reference script hash: "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3"
{
    "marlowe": {
        "hash": "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0",
        "txIn": "aee0ce9972cd036f9088f75ba6602fa03758a3a2a3e48b4713dcda4b815c540d#1"
    },
    "payout": {
        "hash": "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3",
        "txIn": "aee0ce9972cd036f9088f75ba6602fa03758a3a2a3e48b4713dcda4b815c540d#2"
    }
}


## Design the Contract

Set the parameters.

In [40]:
PRINCIPAL=$((20 * ADA))
INTEREST=$((3 * ADA))

GUARANTOR_DEPOSIT_DEADLINE=$((NOW + 1 * DAY))

LENDER_DEPOSIT_DEADLINE=$((NOW + 2 * DAY))

BORROWER_FIRST_DEADLINE=$((NOW + 5 * DAY))
BORROWER_SECOND_DEADLINE=$((NOW + 10 * DAY))
BORROWER_THIRD_DEADLINE=$((NOW + 15 * DAY))

Edit the parameters in the downloaded JSON to instantiate the contract.

In [41]:
sed -e "s/GUARANTOR_ROLE/$GUARANTOR_ROLE/g" \
    -e "s/LENDER_ROLE/$LENDER_ROLE/g" \
    -e "s/BORROWER_ROLE/$BORROWER_ROLE/g" \
    -e "s/PRINCIPAL/$PRINCIPAL/g" \
    -e "s/INTEREST/$INTEREST/g" \
    -e "s/GUARANTOR_DEPOSIT_DEADLINE/$GUARANTOR_DEPOSIT_DEADLINE/g" \
    -e "s/LENDER_DEPOSIT_DEADLINE/$LENDER_DEPOSIT_DEADLINE/g" \
    -e "s/BORROWER_FIRST_DEADLINE/$BORROWER_FIRST_DEADLINE/g" \
    -e "s/BORROWER_SECOND_DEADLINE/$BORROWER_SECOND_DEADLINE/g" \
    -e "s/BORROWER_THIRD_DEADLINE/$BORROWER_THIRD_DEADLINE/g" \
    coupon-bond-guaranteed.json \
    > coupon-bond-guaranteed-1.contract

Start the contract with 2 Ada from the guarantor.

In [42]:
yaml2json << EOI > coupon-bond-guaranteed-1.state
accounts:
- - - role_token: $GUARANTOR_ROLE
    - currency_symbol: ''
      token_name: ''
  - $((2 * ADA))
boundValues: []
choices: []
minTime: $NOW
EOI

## Transaction 1. Create the Contract

Initialize the first `.marlowe` file.

In [43]:
marlowe-cli run initialize "${MAGIC[@]}" \
                           --roles-currency "$ROLES_CURRENCY" \
                           --contract-file coupon-bond-guaranteed-1.contract \
                           --state-file coupon-bond-guaranteed-1.state \
                           --out-file coupon-bond-guaranteed-1.marlowe \
                           --merkleize \
                           --at-address "$FAUCET_ADDR" \
                           --print-stats


Searching for reference script at address: addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h

Expected reference script hash: "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0"

Searching for reference script at address: addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h

Expected reference script hash: "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3"

Validator size: 12505
Base-validator cost: ExBudget {exBudgetCPU = ExCPU 18515100, exBudgetMemory = ExMemory 80600}


The min-Ada for creation belongs to the guarantor.

In [44]:
show-accounts 1

- - - role_token: Guarantor
    - currency_symbol: ''
      token_name: ''
  - 2000000


Submit the transaction.

In [45]:
TX_1=$(
marlowe-cli run auto-execute "${MAGIC[@]}" \
                             --marlowe-out-file coupon-bond-guaranteed-1.marlowe \
                             --required-signer "$GUARANTOR_SKEY" \
                             --change-address "$GUARANTOR_ADDR" \
                             --out-file /dev/null \
                             --print-stats \
                             --submit=600 \
)
echo "TX_1 = $TX_1"


Fee: Lovelace 191681
Size: 479 / 16384 = 2%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%
TX_1 = 95e04eda3a6e86255320c7ea2c54eb05691f81fedfa8d29eacf6caf38196c29c


The Marlowe script address now holds the contract.

In [46]:
query-marlowe "$TX_1"


UTxOs at the contracts's address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
95e04eda3a6e86255320c7ea2c54eb05691f81fedfa8d29eacf6caf38196c29c     1        2000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "02fc4651b13718293b00e0312f9da32aba77ef62fd32761a653317554e2b58b8"


## Transaction 2. Guarantor Desposits

The guarantor must deposit the principal plus all three interest payments.

In [47]:
show-contract 1

timeout: 1664797347000
timeout_continuation: close
when:
- case:
    deposits:
      add:
        multiply: 3
        times: 3000000
      and: 20000000
    into_account:
      role_token: Lender
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      role_token: Guarantor
  merkleized_then: a0bbe8285cd863b56629cac690f526aaa1ddc8a7b203d012a8ec8220b697a123


In [48]:
marlowe-cli run prepare --deposit-party "$GUARANTOR_ROLE" \
                        --deposit-account "$LENDER_ROLE" \
                        --deposit-amount "$((PRINCIPAL + 3 * INTEREST))" \
                        --invalid-before "$(invalid-before 1)" \
                        --invalid-hereafter "$(invalid-hereafter 1)" \
                        --marlowe-file coupon-bond-guaranteed-1.marlowe \
                        --out-file     coupon-bond-guaranteed-2.marlowe \
                        --print-stats


Datum size: 280


Now the lender has the guarantee in their internal account.

In [49]:
show-accounts 2

- - - role_token: Guarantor
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: Lender
    - currency_symbol: ''
      token_name: ''
  - 29000000


Submit the transaction.

In [50]:
TX_2=$(
marlowe-cli run auto-execute "${MAGIC[@]}" \
                             --tx-in-marlowe "$TX_1#1" \
                             --marlowe-in-file coupon-bond-guaranteed-1.marlowe \
                             --marlowe-out-file coupon-bond-guaranteed-2.marlowe \
                             --required-signer "$GUARANTOR_SKEY" \
                             --change-address "$GUARANTOR_ADDR" \
                             --out-file /dev/null \
                             --print-stats \
                             --submit=600 \
)
echo "TX_2 = $TX_2"


Fee: Lovelace 780564
Size: 1301 / 16384 = 7%
Execution units:
  Memory: 7074470 / 14000000 = 50%
  Steps: 1941550605 / 10000000000 = 19%
TX_2 = 00d1cd8e0a8e83d9b8be1adca9946264a3c9e7e524279ab2c3e44b19497a9caf


The Marlowe script address received the deposit.

In [51]:
query-marlowe "$TX_2"


UTxOs at the contracts's address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
00d1cd8e0a8e83d9b8be1adca9946264a3c9e7e524279ab2c3e44b19497a9caf     1        31000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "5b6260fc6ea6b021c8e85400845b8bd5f4b21b819381bc910a79398cbf001f3d"


## Transaction 3. Lender Deposits

The lender must deposit the principal.

In [52]:
show-contract 2

timeout: 1664883747000
timeout_continuation:
  from_account:
    role_token: Lender
  pay:
    add:
      multiply: 3
      times: 3000000
    and: 20000000
  then: close
  to:
    party:
      role_token: Guarantor
  token:
    currency_symbol: ''
    token_name: ''
when:
- case:
    deposits: 20000000
    into_account:
      role_token: Borrower
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      role_token: Lender
  merkleized_then: 3f1aef7a34821b85f97f1d7fb6011295b4fb25226b7d050f63c9aa55f90d0d8b


In [53]:
marlowe-cli run prepare --deposit-party "$LENDER_ROLE" \
                        --deposit-account "$BORROWER_ROLE" \
                        --deposit-amount "$PRINCIPAL" \
                        --invalid-before "$(invalid-before 2)" \
                        --invalid-hereafter "$(invalid-hereafter 2)" \
                        --marlowe-file coupon-bond-guaranteed-2.marlowe \
                        --out-file     coupon-bond-guaranteed-3.marlowe \
                        --print-stats


Datum size: 210
Payment 1
  Acccount: "Borrower"
  Payee: Party "Borrower"
  Ada: Lovelace {getLovelace = 20000000}


The internal accounts are unchanged.

In [54]:
show-accounts 3

- - - role_token: Guarantor
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: Lender
    - currency_symbol: ''
      token_name: ''
  - 29000000


Submit the transaction.

In [55]:
TX_3=$(
marlowe-cli run auto-execute "${MAGIC[@]}" \
                             --tx-in-marlowe "$TX_2#1" \
                             --marlowe-in-file coupon-bond-guaranteed-2.marlowe \
                             --marlowe-out-file coupon-bond-guaranteed-3.marlowe \
                             --required-signer "$LENDER_SKEY" \
                             --change-address "$LENDER_ADDR" \
                             --out-file /dev/null \
                             --print-stats \
                             --submit=600 \
)
echo "TX_3 = $TX_3"


Fee: Lovelace 918091
Size: 1398 / 16384 = 8%
Execution units:
  Memory: 8864408 / 14000000 = 63%
  Steps: 2357350228 / 10000000000 = 23%
TX_3 = fe9a23033ab42aec4de6e50f9e68caa1b6553b21b0c37fb9371125cb9b4cec1a


The Marlowe script address's value is unchanged.

In [56]:
query-marlowe "$TX_3"


UTxOs at the contracts's address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
fe9a23033ab42aec4de6e50f9e68caa1b6553b21b0c37fb9371125cb9b4cec1a     1        31000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "657cbe4c2809a898f910b7e0d4afce975cf2d8f81b07cfafab2a5cc8ab681d45"


But the payout address has the principal ready for the borrower.

In [57]:
query-payout "$TX_3"


UTxOs at the payout address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
fe9a23033ab42aec4de6e50f9e68caa1b6553b21b0c37fb9371125cb9b4cec1a     2        20000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "b4163717b0636a5a3f3e71240389ae94881c4c1563ddf37a6b4c8e7164891acc"


## Transaction 4. Borrower Withdraws Their Loan

The borrower uses their role token to withdraw the principal.

In [58]:
marlowe-cli run auto-withdraw "${MAGIC[@]}" \
                              --marlowe-file coupon-bond-guaranteed-3.marlowe \
                              --role-name "$BORROWER_ROLE" \
                              --change-address "$BORROWER_ADDR" \
                              --required-signer "$BORROWER_SKEY" \
                              --out-file /dev/null \
                              --print-stats \
                              --submit 600


Fee: Lovelace 330306
Size: 450 / 16384 = 2%
Execution units:
  Memory: 1738112 / 14000000 = 12%
  Steps: 486538106 / 10000000000 = 4%
a77b0e21f2cae3c35938e83027c08dfcf06ce57131169dc57df6c10b7a998293


The borrower has received the principal.

In [59]:
query-borrower


UTxOs at the borrower's address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
a77b0e21f2cae3c35938e83027c08dfcf06ce57131169dc57df6c10b7a998293     0        31669694 lovelace + TxOutDatumNone
a77b0e21f2cae3c35938e83027c08dfcf06ce57131169dc57df6c10b7a998293     1        20000000 lovelace + 1 346ec66c37f28ef21ee262eaf41dfe3605aa3344dd236a6003cb606e.426f72726f776572 + TxOutDatumNone


## Transaction 5. Borrower Make First Deposit of Interest

Time passes and the borrower must make their first interest payment.

In [60]:
show-contract 3

timeout: 1665142947000
timeout_continuation: close
when:
- case:
    deposits: 3000000
    into_account:
      role_token: Lender
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      role_token: Borrower
  merkleized_then: c726e2743429c79e222c15f612dd98acd1a342df666e653c782866167ddf057d


This triggers a payment to the lender and a refund of part of the guarantee to the guarantor.

In [61]:
marlowe-cli run prepare --deposit-party "$BORROWER_ROLE" \
                        --deposit-account "$LENDER_ROLE" \
                        --deposit-amount "$INTEREST" \
                        --invalid-before "$(invalid-before 3)" \
                        --invalid-hereafter "$(invalid-hereafter 3)" \
                        --marlowe-file coupon-bond-guaranteed-3.marlowe \
                        --out-file     coupon-bond-guaranteed-4.marlowe \
                        --print-stats


Datum size: 210
Payment 1
  Acccount: "Lender"
  Payee: Party "Lender"
  Ada: Lovelace {getLovelace = 3000000}
Payment 2
  Acccount: "Lender"
  Payee: Party "Guarantor"
  Ada: Lovelace {getLovelace = 3000000}


The internal accounts contain sufficient funds to guarantee the rest of the loan.

In [62]:
show-accounts 4

- - - role_token: Guarantor
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: Lender
    - currency_symbol: ''
      token_name: ''
  - 26000000


Submit the transaction.

In [63]:
TX_4=$(
marlowe-cli run auto-execute "${MAGIC[@]}" \
                             --tx-in-marlowe "$TX_3#1" \
                             --marlowe-in-file coupon-bond-guaranteed-3.marlowe \
                             --marlowe-out-file coupon-bond-guaranteed-4.marlowe \
                             --required-signer "$BORROWER_SKEY" \
                             --change-address "$BORROWER_ADDR" \
                             --out-file /dev/null \
                             --print-stats \
                             --submit=600 \
)
echo "TX_4 = $TX_4"


Fee: Lovelace 1046985
Size: 1456 / 16384 = 8%
Execution units:
  Memory: 10575114 / 14000000 = 75%
  Steps: 2802270124 / 10000000000 = 28%
TX_4 = 27384cd91e10fec5b4496356d3686a12b9d43cbc02652a834fd0181803167a11


The Marlowe script address's value changed because the guarantor received a partial refund.

In [64]:
query-marlowe "$TX_4"


UTxOs at the contracts's address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
27384cd91e10fec5b4496356d3686a12b9d43cbc02652a834fd0181803167a11     1        28000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "e4dc7f73b604920fb2864fb93603e2541fea5b99e2405daf038a98eb62312016"


But the payout address has the funds ready for the lender and guarantor.

In [65]:
query-payout "$TX_4"


UTxOs at the payout address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
27384cd91e10fec5b4496356d3686a12b9d43cbc02652a834fd0181803167a11     2        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "9a46b584520d3445db4a18cefdb256a06f2c0abf1d95ebe523465e71ef1a0942"
27384cd91e10fec5b4496356d3686a12b9d43cbc02652a834fd0181803167a11     3        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "963186f2154ce9c879372b6c7f7122c9bf23386e5ec1be66386256ef25023b6a"


The guarantor and lender will withdraw these funds later.

## Transaction 6. Borrower Make Second Deposit of Interest

Time passes and the borrower must make their second interest payment.

In [66]:
show-contract 5

timeout: 1666004586000
timeout_continuation: close
when:
- case:
    deposits:
      add: 3000000
      and: 20000000
    into_account:
      role_token: Lender
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      role_token: Borrower
  merkleized_then: ed2c966dac497fb1c664dc566ae8623e4edc0ce8f98b1ba71dfd14b4980f6b2d


This triggers a payment to the lender and a refund of part of the guarantee to the guarantor.

In [67]:
marlowe-cli run prepare --deposit-party "$BORROWER_ROLE" \
                        --deposit-account "$LENDER_ROLE" \
                        --deposit-amount "$INTEREST" \
                        --invalid-before "$(invalid-before 4)" \
                        --invalid-hereafter "$(invalid-hereafter 4)" \
                        --marlowe-file coupon-bond-guaranteed-4.marlowe \
                        --out-file     coupon-bond-guaranteed-5.marlowe \
                        --print-stats


Datum size: 223
Payment 1
  Acccount: "Lender"
  Payee: Party "Lender"
  Ada: Lovelace {getLovelace = 3000000}
Payment 2
  Acccount: "Lender"
  Payee: Party "Guarantor"
  Ada: Lovelace {getLovelace = 3000000}


The internal accounts contain sufficient funds to guarantee the rest of the loan.

In [68]:
show-accounts 5

- - - role_token: Guarantor
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: Lender
    - currency_symbol: ''
      token_name: ''
  - 23000000


Submit the transaction.

In [69]:
TX_5=$(
marlowe-cli run auto-execute "${MAGIC[@]}" \
                             --tx-in-marlowe "$TX_4#1" \
                             --marlowe-in-file coupon-bond-guaranteed-4.marlowe \
                             --marlowe-out-file coupon-bond-guaranteed-5.marlowe \
                             --required-signer "$BORROWER_SKEY" \
                             --change-address "$BORROWER_ADDR" \
                             --out-file /dev/null \
                             --print-stats \
                             --submit=600 \
)
echo "TX_5 = $TX_5"


Fee: Lovelace 1148884
Size: 1518 / 16384 = 9%
Execution units:
  Memory: 11801560 / 14000000 = 84%
  Steps: 3134591882 / 10000000000 = 31%
TX_5 = 14d7543fdf706a103dd4df55406a25ba43e99044f05f743408f199fefb6493ba


The Marlowe script address's value changed because the guarantor received a partial refund.

In [70]:
query-marlowe "$TX_5"


UTxOs at the contracts's address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
14d7543fdf706a103dd4df55406a25ba43e99044f05f743408f199fefb6493ba     1        25000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "1a9d1374a9d4d1ff76e0cb9a73752865c5dc7ed23c39997755e088fcc2873f13"


But the payout address has additional funds ready for the lender and guarantor.

In [71]:
query-payout "$TX_5"


UTxOs at the payout address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
14d7543fdf706a103dd4df55406a25ba43e99044f05f743408f199fefb6493ba     2        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "9a46b584520d3445db4a18cefdb256a06f2c0abf1d95ebe523465e71ef1a0942"
14d7543fdf706a103dd4df55406a25ba43e99044f05f743408f199fefb6493ba     3        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "963186f2154ce9c879372b6c7f7122c9bf23386e5ec1be66386256ef25023b6a"


The guarantor and lender will withdraw these funds later.

## Transaction 7. Borrower Make Third Deposit of Interest and Principal

More time passes and the borrower must make their third deposit of interest plus the principal.

In [72]:
show-contract 5

timeout: 1666006947000
timeout_continuation: close
when:
- case:
    deposits:
      add: 3000000
      and: 20000000
    into_account:
      role_token: Lender
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      role_token: Borrower
  merkleized_then: ed2c966dac497fb1c664dc566ae8623e4edc0ce8f98b1ba71dfd14b4980f6b2d


This triggers a payment to the lender and a refund of rest of the guarantee to the guarantor. The guarantor also receives their initial min-Ada from when they created the contract.



In [73]:
marlowe-cli run prepare --deposit-party "$BORROWER_ROLE" \
                        --deposit-account "$LENDER_ROLE" \
                        --deposit-amount "$((PRINCIPAL + INTEREST))" \
                        --invalid-before "$(invalid-before 5)" \
                        --invalid-hereafter "$(invalid-hereafter 5)" \
                        --marlowe-file coupon-bond-guaranteed-5.marlowe \
                        --out-file     coupon-bond-guaranteed-6.marlowe \
                        --print-stats


Datum size: 59
Payment 1
  Acccount: "Lender"
  Payee: Party "Lender"
  Ada: Lovelace {getLovelace = 23000000}
Payment 2
  Acccount: "Lender"
  Payee: Party "Guarantor"
  Ada: Lovelace {getLovelace = 23000000}
Payment 3
  Acccount: "Guarantor"
  Payee: Party "Guarantor"
  Ada: Lovelace {getLovelace = 2000000}


The contract has closed, and it has not funds in internal accounts.

In [74]:
show-contract 6

close
...


In [75]:
show-accounts 6

[]


Submit the transaction.

In [76]:
TX_6=$(
marlowe-cli run auto-execute "${MAGIC[@]}" \
                             --tx-in-marlowe "$TX_5#1" \
                             --marlowe-in-file coupon-bond-guaranteed-5.marlowe \
                             --marlowe-out-file coupon-bond-guaranteed-6.marlowe \
                             --required-signer "$BORROWER_SKEY" \
                             --change-address "$BORROWER_ADDR" \
                             --out-file /dev/null \
                             --print-stats \
                             --submit=600 \
)
echo "TX_6 = $TX_6"


Fee: Lovelace 919910
Size: 1151 / 16384 = 7%
Execution units:
  Memory: 9039188 / 14000000 = 64%
  Steps: 2393448285 / 10000000000 = 23%
TX_6 = f488a476d5e75ca4f715143ea0a668fd2a0a9a70b19fc6b994cb9628896f698e


The Marlowe script address has no UTxO because the contract closed.

In [77]:
query-marlowe "$TX_6"


UTxOs at the contracts's address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


But the payout address has additional funds ready for the lender and guarantor.

In [78]:
query-payout "$TX_6"


UTxOs at the payout address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
f488a476d5e75ca4f715143ea0a668fd2a0a9a70b19fc6b994cb9628896f698e     1        25000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "9a46b584520d3445db4a18cefdb256a06f2c0abf1d95ebe523465e71ef1a0942"
f488a476d5e75ca4f715143ea0a668fd2a0a9a70b19fc6b994cb9628896f698e     2        23000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "963186f2154ce9c879372b6c7f7122c9bf23386e5ec1be66386256ef25023b6a"


View the UTxO's at the borrower's address.

In [79]:
query-borrower


UTxOs at the borrower's address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
14d7543fdf706a103dd4df55406a25ba43e99044f05f743408f199fefb6493ba     0        7761111 lovelace + TxOutDatumNone
14d7543fdf706a103dd4df55406a25ba43e99044f05f743408f199fefb6493ba     5        1500000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "1e5fcd9472a2b1430759b045ae6a8b1f72c9518feadb13dcb9f531c781399aea"
27384cd91e10fec5b4496356d3686a12b9d43cbc02652a834fd0181803167a11     5        1500000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "c726e2743429c79e222c15f612dd98acd1a342df666e653c782866167ddf057d"
f488a476d5e75ca4f715143ea0a668fd2a0a9a70b19fc6b994cb9628896f698e     0        6249784 lovelace + TxOutDatumNone
f488a476d5e75ca4f715143ea0a668fd2a0a9a70b19fc6b994cb9628896f698e     3        1043020 lovelace + 1 346ec66c37f28ef21ee262eaf41dfe3605aa3344dd236a6003cb606e.426f72726f

## Transaction 8. Lender Withdraws the Principal and Interest

The lender uses their role token to withdraw the principal and interest.

In [80]:
marlowe-cli run auto-withdraw "${MAGIC[@]}" \
                              --marlowe-file coupon-bond-guaranteed-6.marlowe \
                              --role-name "$LENDER_ROLE" \
                              --change-address "$LENDER_ADDR" \
                              --required-signer "$LENDER_SKEY" \
                              --out-file /dev/null \
                              --print-stats \
                              --submit 600


Fee: Lovelace 801760
Size: 552 / 16384 = 3%
Execution units:
  Memory: 7776804 / 14000000 = 55%
  Steps: 2130549528 / 10000000000 = 21%
55b599cc47dcea2ef923a250d153d09b362cc4252a1dc37d0b40cde232a0d5c1


The lender has received the principal and interest.

In [81]:
query-lender


UTxOs at the lender's address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
55b599cc47dcea2ef923a250d153d09b362cc4252a1dc37d0b40cde232a0d5c1     0        18780149 lovelace + TxOutDatumNone
55b599cc47dcea2ef923a250d153d09b362cc4252a1dc37d0b40cde232a0d5c1     1        29000000 lovelace + 1 346ec66c37f28ef21ee262eaf41dfe3605aa3344dd236a6003cb606e.4c656e646572 + TxOutDatumNone
fe9a23033ab42aec4de6e50f9e68caa1b6553b21b0c37fb9371125cb9b4cec1a     4        1500000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "3f1aef7a34821b85f97f1d7fb6011295b4fb25226b7d050f63c9aa55f90d0d8b"


## Transaction 9. Guarantor Withdraws Their Guarantee

The guarantor uses their role token to withdraw the guarantee.

In [82]:
marlowe-cli run auto-withdraw "${MAGIC[@]}" \
                              --marlowe-file coupon-bond-guaranteed-6.marlowe \
                              --role-name "$GUARANTOR_ROLE" \
                              --change-address "$GUARANTOR_ADDR" \
                              --required-signer "$GUARANTOR_SKEY" \
                              --out-file /dev/null \
                              --print-stats \
                              --submit 600


Fee: Lovelace 757070
Size: 558 / 16384 = 3%
Execution units:
  Memory: 7174380 / 14000000 = 51%
  Steps: 1989162939 / 10000000000 = 19%
17c121df1218788fd478f5ca225b7343f20d0fc3a79d48f90ef23ae8461460e8


The guarantor has received their guarantee.

In [83]:
query-guarantor


UTxOs at the guarantor's address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
00d1cd8e0a8e83d9b8be1adca9946264a3c9e7e524279ab2c3e44b19497a9caf     3        1500000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "a0bbe8285cd863b56629cac690f526aaa1ddc8a7b203d012a8ec8220b697a123"
17c121df1218788fd478f5ca225b7343f20d0fc3a79d48f90ef23ae8461460e8     0        11404263 lovelace + TxOutDatumNone
17c121df1218788fd478f5ca225b7343f20d0fc3a79d48f90ef23ae8461460e8     1        31000000 lovelace + 1 346ec66c37f28ef21ee262eaf41dfe3605aa3344dd236a6003cb606e.47756172616e746f72 + TxOutDatumNone


## Clean Up

Send the lender's role token to the guarantor and their lovelace back to the faucet.

In [84]:
marlowe-cli util clean "${MAGIC[@]}" \
                       --change-address "$LENDER_ADDR" \
                       --required-signer "$LENDER_SKEY" \
                       --out-file /dev/null \
                       --submit 600

93ce2b3be52a4e6e7ddf1eb4c8c439d20cf60f918f9f5736114abacc0f20adbe


In [85]:
marlowe-cli transaction simple "${MAGIC[@]}" \
                               --tx-in "$(lovelace-utxo $LENDER_ADDR $((2*ADA)))" \
                               --tx-in "$(role-token-utxo $LENDER_ADDR $LENDER_TOKEN)" \
                               --tx-out "$GUARANTOR_ADDR+$((2*ADA))+1 $LENDER_TOKEN" \
                               --change-address "$FAUCET_ADDR" \
                               --required-signer "$LENDER_SKEY" \
                               --out-file /dev/null \
                               --submit 600

cde2f3f0c3e8972237c498144ea1ec2ee0a2323c658480141d70edddf92593ab


In [86]:
query-lender


UTxOs at the lender's address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


Send the borrower's role token and funds to the guarantor and their lovelace back to the faucet.

In [87]:
marlowe-cli util clean "${MAGIC[@]}" \
                       --change-address "$BORROWER_ADDR" \
                       --required-signer "$BORROWER_SKEY" \
                       --out-file /dev/null \
                       --submit 600

7ed785f2b16e87394ec07dacf1b023ec55c961f96d1e621689c243950aec75ac


In [88]:
marlowe-cli transaction simple "${MAGIC[@]}" \
                               --tx-in "$(lovelace-utxo $BORROWER_ADDR $((2*ADA)))" \
                               --tx-in "$(role-token-utxo $BORROWER_ADDR $BORROWER_TOKEN)" \
                               --tx-out "$GUARANTOR_ADDR+$((2*ADA))+1 $BORROWER_TOKEN" \
                               --change-address "$FAUCET_ADDR" \
                               --required-signer "$BORROWER_SKEY" \
                               --out-file /dev/null \
                               --submit 600

53bbdc1f6a8fed9d892f9695a8fa3a4ec59f9e809e20aff9d7313ecc74e4bc0a


In [89]:
query-borrower


UTxOs at the borrower's address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


Burn the tokens and send the lovelace to back to the faucet.

In [90]:
marlowe-cli util burn "${MAGIC[@]}" \
                      --expires "$MINT_EXPIRES" \
                      --issuer "$GUARANTOR_ADDR:$GUARANTOR_SKEY" \
                      --token-provider "$GUARANTOR_ADDR:$GUARANTOR_SKEY" \
                      --out-file /dev/null \
                      --submit 600


Fee: Lovelace 194849
Size: 351 / 16384 = 2%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%
346ec66c37f28ef21ee262eaf41dfe3605aa3344dd236a6003cb606e


In [91]:
marlowe-cli transaction simple "${MAGIC[@]}" \
                               --tx-in "$(lovelace-utxo $GUARANTOR_ADDR 1)" \
                               --change-address "$FAUCET_ADDR" \
                               --required-signer "$GUARANTOR_SKEY" \
                               --out-file /dev/null \
                               --submit 600

b5e097f93770410df7f3400a3fb56529d6a0f7db28aee847a813af546d381788


In [92]:
query-guarantor


UTxOs at the guarantor's address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


View the UTxOs at the faucet.

In [93]:
marlowe-cli util clean "${MAGIC[@]}" \
                       --change-address "$FAUCET_ADDR" \
                       --required-signer "$FAUCET_SKEY" \
                       --out-file /dev/null \
                       --submit 600

d86c2ed60bcf8c5a63262037e6507e53acbe73bff7c57fc8134211bffb66b5a1


In [94]:
cardano-cli query utxo "${MAGIC[@]}" --address "$FAUCET_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
d86c2ed60bcf8c5a63262037e6507e53acbe73bff7c57fc8134211bffb66b5a1     0        230122886 lovelace + TxOutDatumNone


## Total Cost in Fees

Compare the funds in the faucet at the beginning of the transactions to the funds in the faucet at the end of the transactions.

In [95]:
echo "The faucet started with $(echo scale=6\;$FAUCET_INITIAL_FUNDS/1000000 | bc) Ada."

FAUCET_FINAL_FUNDS=$(query-faucet)
echo "The faucet ended with $(echo scale=6\;$FAUCET_FINAL_FUNDS/1000000 | bc) Ada."

echo "The total cost of the transactions was $(echo scale=6\;\($FAUCET_INITIAL_FUNDS-$FAUCET_FINAL_FUNDS\)/1000000 | bc) Ada."

The faucet started with 240.018586 Ada.
The faucet ended with 230.122886 Ada.
The total cost of the transactions was 9.895700 Ada.
